In [1]:
from strategy.crypto_seven import CryptoSeven
from database.sec import SEC
from database.market import Market
from modeler.modeler import Modeler as sp
from preprocessor.model_preprocessor import ModelPreprocessor as mp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math

In [2]:
## Loading Constants
start = "2014-08-01"
end = datetime(2020,10,1).strftime("%Y-%m-%d")
# Loading Databases)
market = Market()
strat = CryptoSeven()
weekly_range = range(1,54)
yearly_range = range(2015,2021)

In [24]:
btc = pd.read_csv("BTC-USD.csv")
eth = pd.read_csv("ETH-USD.csv")

In [25]:
for column in btc.columns:
    new_column = column.strip().replace(" ","_").lower().replace("(usd)","")
    btc.rename(columns={column:new_column},inplace=True)
for column in eth.columns:
    new_column = column.strip().replace(" ","_").lower().replace("(usd)","")
    eth.rename(columns={column:new_column},inplace=True)

In [26]:
## btc week conversions
btc["date"] = pd.to_datetime(btc["date"])
eth["date"] = pd.to_datetime(eth["date"])
btc["quarter"] = [row[1]["date"].quarter for row in btc.iterrows()]
btc["week"] = [row[1]["date"].week for row in btc.iterrows()]
btc["year"] = [row[1]["date"].year for row in btc.iterrows()]
eth["quarter"] = [row[1]["date"].quarter for row in eth.iterrows()]
eth["week"] = [row[1]["date"].week for row in eth.iterrows()]
eth["year"] = [row[1]["date"].year for row in eth.iterrows()]
btc.rename(columns={"adj_close":"closing_price_"},inplace=True)
eth.rename(columns={"adj_close":"closing_price_"},inplace=True)
btc["rolling_7"] = btc["closing_price_"].rolling(window=7).mean()
btc["rolling_30"] = btc["closing_price_"].rolling(window=30).mean()
btc["rolling_100"] = btc["closing_price_"].rolling(window=100).mean()
eth["rolling_7"] = eth["closing_price_"].rolling(window=7).mean()
eth["rolling_30"] = eth["closing_price_"].rolling(window=30).mean()
eth["rolling_100"] = eth["closing_price_"].rolling(window=100).mean()
for i in range(1,6):
    btc[f'price_{i}'] = btc["closing_price_"].shift(i)
    eth[f'price_{i}'] = eth["closing_price_"].shift(i)
btc.drop(["open","high","low"],axis=1,inplace=True)
eth.drop(["open","high","low"],axis=1,inplace=True)

In [27]:
btc = btc.iloc[100:]
eth = eth.iloc[100:]

In [28]:
datasets = {"btc":btc,"eth":eth}

In [29]:
training_days = 100
daily_gap = 2
for year in tqdm(yearly_range):
    for week in tqdm(weekly_range):
        for dataset in datasets:
            ds = datasets[dataset]
            if ds[(ds["year"]==year) & (ds["week"]==week)].index.size == 0:
                continue
            else
                try:
                    training_data, prediction_data = strat.transform(ds.copy(),year,week,daily_gap,training_days)
                    refined_data = mp.preprocess(training_data)
                    regression_models = sp.regression(refined_data.copy(),ranked=False,tf=False,deep=False)
                    classification_models = sp.classification(refined_data.copy(),tf=False,deep=False)
                    models = pd.DataFrame([regression_models,classification_models])
                    factors = refined_data["X"].columns
                    sim = strat.create_sim(dataset,models,factors,prediction_data)
                except Exception as e:
                    print(year,week,str(e))

 74%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                         | 39/53 [01:03<00:24,  1.75s/it]


 87%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                    | 46/53 [01:15<00:11,  1.68s/it]

2015 46 division by zero



 60%|█████████████████████████████████████████████████████████████████████████████████████████████▌                                                             | 32/53 [01:40<01:05,  3.14s/it]


 34%|████████████████████████████████████████████████████▋                                                                                                      | 18/53 [00:59<01:53,  3.24s/it]


...


 85%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                       | 45/53 [02:21<00:25,  3.13s/it]

2020 46 Input contains NaN, infinity or a value too large for dtype('float64').



 87%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                    | 46/53 [02:24<00:22,  3.16s/it]

2020 46 Input contains NaN, infinity or a value too large for dtype('float64').



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [15:47<00:00, 157.87s/it]


In [30]:
strat.db.connect()
sim = strat.db.retrieve_data("sim")
strat.db.close()

In [31]:
btc["currency"] = "btc"
eth["currency"] = "eth"
price = pd.concat([btc,eth])

In [32]:
sim = sim.merge(price[["date","currency","closing_price_"]],on=["date","currency"],how="left")
sim["predicted_delta"]  = (sim["crypto_regression_prediction"] - sim["closing_price_"]) / sim["closing_price_"]

In [33]:
sim.drop("_id",axis=1,inplace=True)

In [34]:
sim.columns

Index(['date', 'crypto_regression_prediction', 'crypto_regression_score',
       'crypto_classification_prediction', 'crypto_classification_score',
       'currency', 'closing_price_', 'predicted_delta'],
      dtype='object')

In [35]:
seats = 11
yearly_delta_range = range(100,1000,100)
ceiling_settings = [True,False]
weekly_value_settings = [True,False]
trade_signal_score_settings = range(60,90,10)
start = datetime(2015,1,1)
end = datetime(2020,12,24)
initial = 100

In [36]:
epoch = 0
seats = 1
strat.db.connect()
strat.db.drop_table("epochs")
iterations = len(yearly_delta_range) * len(ceiling_settings) * len(weekly_value_settings) * len(trade_signal_score_settings)
print(iterations)
for i in range(iterations):
    strat.db.drop_table("trades_{}".format(i))
for ceiling in tqdm(ceiling_settings):
    for weekly_value in tqdm(weekly_value_settings):
        for delta in tqdm(yearly_delta_range):
            for score in tqdm(trade_signal_score_settings):
                s = float(score/100)
                dr = float(delta/100)
                print(s,dr)
                epoch_dict = {"epoch":epoch
                              ,"dr":dr
                              ,"ts_score":s
                              ,"ceiling":ceiling
                              ,"weekly_value":weekly_value
                             }
                ts = strat.backtest(start,end,sim.copy(),seats,ceiling,weekly_value,dr,s)
                if len(ts) > 0:
                    strat.db.store_data("epochs",pd.DataFrame([epoch_dict]))
                    strat.db.store_data("trades_{}".format(epoch),pd.DataFrame(ts))
                epoch += 1
strat.db.close()

108


  0%|                                                                                                                                                                     | 0/2 [00:00<?, ?it/s]

  0%|                                                                                                                                                                     | 0/9 [00:00<?, ?it/s]


  0%|                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

0.6 1.0





 33%|████████████████████████████████████████████████████▎                                                                                                        | 1/3 [00:13<00:26, 13.42s/it]

0.7 1.0


...




 33%|████████████████████████████████████████████████████▎                                                                                                        | 1/3 [00:09<00:18,  9.28s/it]

0.7 9.0





 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                    | 2/3 [00:14<00:08,  8.06s/it]

0.8 9.0





100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:17<00:00,  5.83s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [02:39<00:00, 17.69s/it]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [11:24<00:00, 342.22s/it]
